## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/3cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,23.16,88,40,4.12,scattered clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,2.81,81,75,0.00,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,4.06,81,0,4.45,clear sky
3,3,Provideniya,RU,64.3833,-173.3000,3.68,63,16,1.80,few clouds
4,4,Fortuna,US,40.5982,-124.1573,18.57,66,75,2.24,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
21,21,Bathsheba,BB,13.2167,-59.5167,30.34,54,40,8.32,scattered clouds
44,44,Puerto Escondido,MX,15.8500,-97.0667,31.59,64,75,3.22,broken clouds
56,56,Nalut,LY,30.3333,10.8500,35.98,10,25,5.33,scattered clouds
57,57,San Patricio,US,28.0170,-97.5169,32.23,64,3,7.40,clear sky
74,74,Bonfim,BR,3.0833,-59.9500,30.45,62,41,2.45,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                87
City                   87
Country                87
Lat                    87
Lng                    87
Max Temp               87
Humidity               87
Cloudiness             87
Wind Speed             87
Current Description    87
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
21,21,Bathsheba,BB,13.2167,-59.5167,30.34,54,40,8.32,scattered clouds
44,44,Puerto Escondido,MX,15.8500,-97.0667,31.59,64,75,3.22,broken clouds
56,56,Nalut,LY,30.3333,10.8500,35.98,10,25,5.33,scattered clouds
57,57,San Patricio,US,28.0170,-97.5169,32.23,64,3,7.40,clear sky
74,74,Bonfim,BR,3.0833,-59.9500,30.45,62,41,2.45,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
668,668,Sena Madureira,BR,-9.0656,-68.6569,32.12,51,31,1.10,scattered clouds
680,680,Saint Andrews,US,34.0454,-81.1293,32.36,65,75,2.24,broken clouds
684,684,Tondon,GN,10.3667,-13.3500,32.20,58,76,1.68,broken clouds
688,688,Keita,NE,14.7551,5.7749,37.56,16,100,4.05,overcast clouds


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
21,Bathsheba,BB,30.34,scattered clouds,13.2167,-59.5167,
44,Puerto Escondido,MX,31.59,broken clouds,15.8500,-97.0667,
56,Nalut,LY,35.98,scattered clouds,30.3333,10.8500,
57,San Patricio,US,32.23,clear sky,28.0170,-97.5169,
74,Bonfim,BR,30.45,scattered clouds,3.0833,-59.9500,
78,Bilma,NE,36.98,overcast clouds,18.6853,12.9164,
82,Avera,US,32.17,overcast clouds,33.1940,-82.5271,
93,Roswell,US,30.03,overcast clouds,34.0232,-84.3616,
110,Albany,US,33.03,broken clouds,42.6001,-73.9662,
111,Urubicha,BO,33.83,clear sky,-15.3833,-62.9500,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found")
        

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("",np.nan,inplace=True)
hotel_df=hotel_df.dropna()

In [29]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))